In [1]:
import time
import paho.mqtt.client as mqtt
from collections import deque
from scipy.spatial.distance import euclidean
from statistics import mode
import ast  # to safely parse the published string back into a dictionary

# Buffer to store the last 5 received records
buffer = deque(maxlen=5)
correct = 0
iterations = 0
max_iterations = 1000

In [2]:
import paho.mqtt.client as mqtt
from collections import deque
from statistics import mode
from scipy.spatial.distance import euclidean
import ast  # to safely convert string to dictionary

# Global variables
buffer = deque(maxlen=5)  # Buffer for the last 5 records
correct = 0
iterations = 0
max_iterations = 1000

# 3NN classifier function (using Euclidean distance on normalized features)
def three_nn_classify(query, records):
    distances = []
    for rec in records:
        dist = euclidean(
            [query['Glucose'], query['BloodPressure'], query['Insulin']],
            [rec['Glucose'], rec['BloodPressure'], rec['Insulin']]
        )
        distances.append((dist, rec['Outcome']))
    distances.sort(key=lambda x: x[0])
    neighbors = distances[:3]
    outcomes = [n[1] for n in neighbors]
    try:
        return mode(outcomes)
    except Exception:
        return outcomes[0]  # Fallback in case of tie

# Callback when connecting to the broker
def on_connect(client, userdata, flags, rc):
    print("Connected with result code " + str(rc))
    client.subscribe("myroom/diabetes")  # subscribe to our topic

# Callback when a message is received
def on_message(client, userdata, msg):
    global correct, iterations
    record_str = msg.payload.decode()
    try:
        # Convert the received string to a dictionary
        record = ast.literal_eval(record_str)
    except Exception as e:
        print("Error decoding record:", e)
        return

    # If we have 5 records in our buffer, classify the new record using 3NN
    if len(buffer) == 5:
        prediction = three_nn_classify(record, list(buffer))
        iterations += 1
        if prediction == record['Outcome']:
            correct += 1
        print(f"Iteration {iterations}: Predicted={prediction}, Actual={record['Outcome']}")
        # Stop after max_iterations are reached
        if iterations >= max_iterations:
            print(f"Completed {max_iterations} iterations with {correct} correct classifications.")
            client.disconnect()
            return

    # Append the current record to the buffer
    buffer.append(record)

# Set up MQTT client
mqttc = mqtt.Client()
mqttc.on_connect = on_connect
mqttc.on_message = on_message

# Connect to the public MQTT broker
mqttc.connect("mqtt.eclipseprojects.io", 1883, 60)

# Start the MQTT client loop (this is blocking)
mqttc.loop_forever()


Connected with result code 0
Iteration 1: Predicted=0, Actual=1
Iteration 2: Predicted=1, Actual=0
Iteration 3: Predicted=0, Actual=1
Iteration 4: Predicted=1, Actual=0
Iteration 5: Predicted=1, Actual=0
Iteration 6: Predicted=0, Actual=1
Iteration 7: Predicted=0, Actual=0
Iteration 8: Predicted=0, Actual=0
Iteration 9: Predicted=0, Actual=1
Iteration 10: Predicted=0, Actual=0
Iteration 11: Predicted=0, Actual=0
Iteration 12: Predicted=0, Actual=0
Iteration 13: Predicted=0, Actual=0
Iteration 14: Predicted=0, Actual=1
Iteration 15: Predicted=0, Actual=1
Iteration 16: Predicted=0, Actual=0
Iteration 17: Predicted=0, Actual=1
Iteration 18: Predicted=0, Actual=0
Iteration 19: Predicted=1, Actual=0
Iteration 20: Predicted=0, Actual=0
Iteration 21: Predicted=0, Actual=0
Iteration 22: Predicted=0, Actual=1
Iteration 23: Predicted=0, Actual=1
Iteration 24: Predicted=0, Actual=0
Iteration 25: Predicted=0, Actual=1
Iteration 26: Predicted=0, Actual=0
Iteration 27: Predicted=0, Actual=0
Iteratio

7